# Response 1

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np

# Download the dataset
path = get_file('alice.txt', origin='https://www.gutenberg.org/files/11/11-0.txt')

# Read the text
text = open(path).read().lower()

# Remove unnecessary characters
text = text.replace('\n', ' ')
text = text.replace('\r', ' ')
text = text.replace('\ufeff', '')

# Create a list of unique characters
chars = sorted(list(set(text)))

# Create dictionaries to map characters to indices and vice versa
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Define the sequence length and step size
maxlen = 40
step = 3

# Create semi-redundant sequences
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

# Create the training data
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Define the model
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

# Compile the model
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X, y, batch_size=256, epochs=4)

# Define a function to sample an index from a probability array
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Generate text
diversity = 0.2
sentence = 'Alice was beginning to get very tired'
generated = sentence
print('...Generating with seed: "' + sentence + '"')
for i in range(100):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    sentence = sentence[1:] + next_char
    generated += next_char
print("...Generated: ", generated)

154638/154638 [==============================] - 1s 4us/step
Epoch 1/4
189/189 [==============================] - 3s 7ms/step - loss: 2.5037 - accuracy: 0.3086
Epoch 2/4
189/189 [==============================] - 1s 7ms/step - loss: 1.9511 - accuracy: 0.4406
Epoch 3/4
189/189 [==============================] - 1s 7ms/step - loss: 1.7434 - accuracy: 0.4900
Epoch 4/4
189/189 [==============================] - 1s 6ms/step - loss: 1.6070 - accuracy: 0.5268
...Generating with seed: "Alice was beginning to get very tired"


KeyError: 'A'

# Response 2

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random

# Download the dataset
path = get_file('alice.txt', origin='https://www.gutenberg.org/files/11/11-0.txt')
text = open(path).read().lower()

# Remove unnecessary characters
text = text.replace('--', ' ')
text = text.replace('\n', ' ')
text = text.replace('"', '')
text = text.replace("'", '')

# Create a set of unique characters
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

# One-hot encode the sequences
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Build the model
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X, y, batch_size=256, epochs=4)

# Define a function to sample an index from a probability array
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Generate text
diversity = 0.2
sentence = 'Alice was beginning to get very tired'
generated = sentence
print('...Generating with seed: "' + sentence + '"')
for i in range(100):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    sentence = sentence[1:] + next_char
    generated += next_char
print("...Generated: ", generated)

Epoch 1/4
189/189 [==============================] - 4s 6ms/step - loss: 2.4685 - accuracy: 0.3134
Epoch 2/4
189/189 [==============================] - 1s 6ms/step - loss: 1.9326 - accuracy: 0.4426
Epoch 3/4
189/189 [==============================] - 1s 6ms/step - loss: 1.7221 - accuracy: 0.4957
Epoch 4/4
189/189 [==============================] - 1s 6ms/step - loss: 1.5827 - accuracy: 0.5333
...Generating with seed: "Alice was beginning to get very tired"


KeyError: 'A'